# Chatbot with Agents and Memory

## Imports

In [1]:
# System
import os
os.environ['USER_AGENT'] = 'JimYin88'

# LLM Models
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama

# Templates
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain import hub


# Document Loaders
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import WebBaseLoader
# from langchain.document_loaders import BSHTMLLoader
# from langchain_community.document_loaders import UnstructuredRTFLoader

# Document Splitters
from langchain.text_splitter import TokenTextSplitter
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_openai import OpenAIEmbeddings

# Vector Stores
from langchain_chroma import Chroma
from langchain_community.vectorstores.faiss import FAISS

# LangChain Chains
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

# Gradio
import gradio as gr

# Tools
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.tools import Tool
from langchain.agents import tool, load_tools, initialize_agent, AgentType
from langchain_community.tools.google_books import GoogleBooksQueryRun
from langchain_community.utilities.google_books import GoogleBooksAPIWrapper
from langchain_community.tools.google_trends import GoogleTrendsQueryRun
from langchain_community.utilities.google_trends import GoogleTrendsAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# History Memory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Text-to-Speech
import elevenlabs
from elevenlabs.client import ElevenLabs

## Load Environment Variables

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Instantiate LLM model

### Using ChatGPT 4o-mini

In [3]:
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_completion_tokens=1028,
                        temperature=0.2)

## Search Tool

In [4]:
# Tavily Search
tavily_search = TavilySearchResults(max_results=2)

# Wikipedia
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Google Trend
google_trend_tool = GoogleTrendsQueryRun(api_wrapper=GoogleTrendsAPIWrapper())

# Google Book
google_book_tool = GoogleBooksQueryRun(api_wrapper=GoogleBooksAPIWrapper())

## Retriever Tool

In [5]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=20)
splitdocs = splitter.split_documents(docs)
embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(documents=splitdocs, embedding=embedding_function)
retriever = vector_store.as_retriever()

In [6]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

## Custom Tool

In [7]:
@tool
def get_stock_closing_price(ticker: str) -> str:
    ''' 
    Returns the stock's closing prices for the past 20 days.
    Expects a string of the company's stock ticker and returns 
    the date and the stock closing prices for the past 20 days.
    '''
    
    alpha_vantage = AlphaVantageAPIWrapper()
    
    daily_price = alpha_vantage._get_time_series_daily(ticker)
    
    result = ''
    
    result += f'Stock Ticker: {ticker}\n\n'
    result += 'Date ' + ' \t       ' + ' Closing Price\n'
    
    counter = 0
    for daily in daily_price['Time Series (Daily)']:
        if counter > 20:
            break
        else:
            result += daily + '\t' + daily_price['Time Series (Daily)'][daily]['4. close'] + '\n'
            counter += 1
    
    return result

## Create a list of tools

In [8]:
# tools = load_tools(tool_names=["serpapi", "wolfram-alpha"], llm=chat_model)
# tools = tools + [retriever_tool]

tools_list = load_tools(tool_names=["wolfram-alpha"], llm=chat_model)
tools_list = tools_list + [tavily_search, retriever_tool]
tools_list = tools_list + [get_stock_closing_price]
tools_list = tools_list + [google_book_tool, google_trend_tool, wikipedia_tool]

In [9]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Create agent

In [10]:
agent = create_tool_calling_agent(llm=chat_model, 
                                  tools=tools_list, 
                                  prompt=prompt)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools_list)

## Define Chatbot_streaming function

In [12]:
history = []

In [13]:
def chat_stream(message, history):
    
    # messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = agent_executor.stream({"input": message, "chat_history": history})

    response = ""
    for chunk in stream:
        if 'output' in chunk:
            response += chunk['output'] or ''
        else:
            continue
        yield response

## Define a Chatbot with voice

In [14]:
client = ElevenLabs(api_key=os.getenv('ELEVENLABS_API_KEY'))

def text_to_speech(text, voice="Daniel"):

    voice_catalog = {"Aria": "9BWtsMINqrJLrRacOk9x",
                     "Roger": "CwhRBWXzGAHq8TQ4Fs17",
                     "Sarah": "EXAVITQu4vr4xnSDxMaL",
                     "Laura": "FGY2WhTYpPnrIDTdsKH5",
                     "Charlie": "IKne3meq5aSn9XLyUdCD",
                     "George": "JBFqnCBsd6RMkjVDRZzb",
                     "Callum": "N2lVS1w4EtoT3dr4eOWO",
                     "River": "SAz9YHcvj6GT2YYXdXww",
                     "Liam": "TX3LPaxmHKxFdv7VOQHJ",
                     "Charlotte": "XB0fDUnXU5powFXDhCwa",
                     "Alice": "Xb7hH8MSUJpSbSDYk0k2",
                     "Matilda": "XrExE9yKIg1WjnnlVkGX",
                     "Will": "bIHbv24MWmeRgasZH58o",
                     "Jessica": "cgSgspJ2msm6clMCkdW9",
                     "Eric": "cjVigY5qzO86Huf0OWal",
                     "Chris": "iP95p4xoKVk53GoZ742B",
                     "Brian": "nPczCjzI2devNBz1zQrb",
                     "Daniel": "onwK4e9ZLuTAKqWW03F9",
                     "Lily": "pFZP5JQG7iQjIQuC4Bku",
                     "Bill": "pqHfZKP75CvOlQylNhV4", 
                     "Stacy - Sweet and Cute Chinese": "hkfHEbBvdQFNX4uWHqRF", 
                     "James Gao": "4VZIsMPtgggwNg7OXbPY"}

    if voice in voice_catalog:
        voice_id_code = voice_catalog[voice]
    else:
        voice_id_code = voice_catalog['Daniel']

    audio = client.text_to_speech.convert(text=text, 
                                          voice_id=voice_id_code,
                                          model_id='eleven_turbo_v2_5',
                                          output_format="mp3_44100_128")

    elevenlabs.play(audio)

In [15]:
def chat_voice(message, history):
    
    # messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = agent_executor.invoke({"input": message, "chat_history": history})
    text_to_speech(text=response['output'], voice="Daniel")

    return response['output']

## Gradio Interface

In [16]:
chatbot = gr.ChatInterface(fn=chat_stream, type="messages")

In [17]:
chatbot.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [18]:
chatbot.close()

Closing server running on port: 7860
